### 문제
- $N \le 200000$ 인 $|N|$ 인 수열 $K$ 에서 중복을 포함하여 숫자 최대 2개를 뽑은 것을 $l$ 이라고 할 때, $\displaystyle\sum^{|l|}_{i=1} l_i \in D$ 인 원소의 개수를 구하는 문제이다.

### 풀이
- $N$ 차 다항식을 만든 뒤 제곱했을 때, 그 다항식의 차수가 $D$ 에 존재할때의 개수를 구하면 된다.
- DFT로 바꾸기 위한 배열의 크기가 2의 제곱수여야함에 주의한다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

rt = [1]
def fft(P):
  n = len(P)
  P = list(P)
  assert n and (n - 1) & n == 0
  
  while 2 * len(rt) < n:
    import cmath
    root = cmath.exp(2j * cmath.pi / (4 * len(rt)))
    rt.extend([r * root for r in rt])

  k = n
  while k > 1:
    for i in range(n//k):
      r = rt[i]
      for j1 in range(i*k, i*k + k//2):
        j2 = j1 + k//2
        z = r * P[j2]
        P[j2] = P[j1] - z
        P[j1] = P[j1] + z
    k //= 2
  
  rev = [0] * n
  for i in range(1, n):
    rev[i] = rev[i // 2] // 2 + (i & 1) * n // 2
  return [P[r] for r in rev]

def ifft(P):
  n = len(P)
  return fft([P[-i]/n for i in range(n)])

def fft_conv(P, Q):
  m = len(P) + len(Q) - 1
  n = 1 << m.bit_length()

  P = P + [0] * (n - len(P))
  Q = Q + [0] * (n - len(Q))
  P, Q = fft(P), fft(Q)

  return ifft([p*q for p,q in zip(P, Q)])[:m]

def sol() :
  N = int(input())
  K = [int(input()) for _ in range(N)]
  M = int(input())
  D = [int(input()) for _ in range(M)]

  f = [0] * (1 << 18)
  f[0] = 1
  for v in K :
    f[v] = 1
  
  ff = fft_conv(f,f) #제곱
  ff = [*map(lambda x: round(x.real), ff)]
  cnt = sum(1 for d in D if ff[d])
  
  print(cnt)

sol()